**Importing libraries**

In [1]:
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')

**Loading dataset**

In [2]:
dfu=pd.read_excel("data_c_1.xlsx", sheet_name='Usage', header=None) # not to convert first row to column name
dfc=pd.read_excel("data_c_1.xlsx", sheet_name='Contracts')
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
dfu.columns=['info']
dfu.head()

,info
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh


**Preprocessing**

In [3]:
# getting kilowatt results
rd=pd.DataFrame(data=None)
rd['watt']=dfu['info'].str.split('_').str.slice(-1).astype(str) # extracting kilowatt per hour
rd['watt']=rd['watt'].str.replace('kwh','').str.replace(' ','').str.slice(2,-2).astype(float)
# rd['watt'].apply(lambda x: float(x)) # to convert to float in another way
rd.head()

,watt
0,0.384
1,1.201
2,1.523
3,0.424
4,0.209


In [4]:
# getting months
rd['month']=dfu['info'].str.split('-').str.slice(-2,-1).astype(str).str.slice(2,-2)
rd.head()

,watt,month
0,0.384,Mar
1,1.201,Aug
2,1.523,Mar
3,0.424,Apr
4,0.209,Dec


In [5]:
# getting time results
rd['time']=dfu['info'].str.replace('AM','AMxx').str.replace('PM','PMxx').str.split('xx').str.slice(0,1).astype(str).str.slice(2,-2)
rd['time']=rd['time'].str.replace('_','').str.replace(' ','')
rd.head()

,watt,month,time
0,0.384,Mar,3PM
1,1.201,Aug,5AM
2,1.523,Mar,8PM
3,0.424,Apr,6PM
4,0.209,Dec,1AM


In [6]:
rd['day']=dfu['info'].str.split('-').str.slice(0,1).astype(str).str.split(' ').str.slice(-1).astype(str).str.slice(2,-4)
rd['day']=rd['day'].str.replace('th','').str.replace('st','').str.replace('nd','').str.replace('rd','').str.replace(' ','').astype(int)
rd.head()

,watt,month,time,day
0,0.384,Mar,3PM,24
1,1.201,Aug,5AM,15
2,1.523,Mar,8PM,20
3,0.424,Apr,6PM,23
4,0.209,Dec,1AM,19


In [7]:
wd={'Mon','Tue','Wed','Thu','Fri','Sat','Sun'}
rd['weekday']=dfu['info'].str.split('-').str.slice(0,1).astype(str).str.split(' ').str.slice(-2,-1).astype(str).str.slice(2,-2)
rd=rd.replace({'Monday','Tuesday','Wednesday','Thursday', 'Friday','Saturday','Sunday'},wd)
def ch(x):
    if str(x) not in wd:
        return np.NaN
    else:
        return x
rd['weekday']=rd['weekday'].apply(ch)
rd.head()

,watt,month,time,day,weekday
0,0.384,Mar,3PM,24,Mon
1,1.201,Aug,5AM,15,NaN
2,1.523,Mar,8PM,20,Thu
3,0.424,Apr,6PM,23,NaN
4,0.209,Dec,1AM,19,Sun


### let's check last time whether there is an abnormal value or not in related columns

In [8]:
print(rd['time'].value_counts()) # no abnormal value
print(rd['month'].value_counts()) # no abnormal value
print(rd['day'].value_counts()) # no abnormal value
print(rd['weekday'].value_counts()) # no abnormal value

10PM    365
1AM     365
8AM     365
7PM     365
4PM     365
3PM     365
9PM     365
4AM     365
2AM     365
5PM     365
11PM    365
11AM    365
8PM     365
5AM     365
6PM     365
3AM     365
2PM     365
12PM    365
12AM    365
9AM     365
1PM     365
6AM     365
7AM     365
10AM    365
Name: time, dtype: int64
Aug    744
Jul    744
Mar    744
Dec    744
May    744
Oct    744
Jan    744
Jun    720
Nov    720
Sep    720
Apr    720
Feb    672
Name: month, dtype: int64
4     288
27    288
16    288
24    288
1     288
9     288
17    288
25    288
2     288
10    288
18    288
26    288
3     288
11    288
19    288
8     288
23    288
12    288
20    288
28    288
5     288
13    288
21    288
6     288
14    288
22    288
7     288
15    288
29    264
30    264
31    168
Name: day, dtype: int64
Tue    674
Fri    647
Wed    642
Mon    633
Thu    628
Sat    624
Sun    611
Name: weekday, dtype: int64


In [9]:
# combining date and time columns to remove duplicates
rd['datetime']=rd['day'].astype(str)+'-'+rd['month']+'-'+rd['time']
rd

,watt,month,time,day,weekday,datetime
0,0.384,Mar,3PM,24,Mon,24-Mar-3PM
1,1.201,Aug,5AM,15,NaN,15-Aug-5AM
2,1.523,Mar,8PM,20,Thu,20-Mar-8PM
3,0.424,Apr,6PM,23,NaN,23-Apr-6PM
4,0.209,Dec,1AM,19,Sun,19-Dec-1AM
5,1.228,Aug,5AM,19,Tue,19-Aug-5AM
6,1.296,Jul,12PM,7,Mon,7-Jul-12PM
7,0.437,Nov,7AM,25,Tue,25-Nov-7AM
8,0.523,Aug,8AM,14,NaN,14-Aug-8AM
9,2.052,Jan,4PM,25,NaN,25-Jan-4PM


In [10]:
rd.duplicated(subset='datetime').value_counts() # no duplicated value!

False    8760
dtype: int64

In [11]:
# our dataframe is ready:)
rd.head()

,watt,month,time,day,weekday,datetime
0,0.384,Mar,3PM,24,Mon,24-Mar-3PM
1,1.201,Aug,5AM,15,NaN,15-Aug-5AM
2,1.523,Mar,8PM,20,Thu,20-Mar-8PM
3,0.424,Apr,6PM,23,NaN,23-Apr-6PM
4,0.209,Dec,1AM,19,Sun,19-Dec-1AM


In [12]:
dfc.drop(index=range(0,8), inplace=True)

In [13]:
dfc

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
8,Flat Rate,0.21,$/kWh,NaN,January,0.2,$/kWh,NaN,12AM until 1AM,0.1,$/kWh
9,NaN,NaN,NaN,NaN,February,0.19,$/kWh,NaN,1AM until 2AM,0.1,$/kWh
10,NaN,NaN,NaN,NaN,March,0.17,$/kWh,NaN,2AM until 3AM,0.1,$/kWh
11,NaN,NaN,NaN,NaN,April,0.18,$/kWh,NaN,3AM until 4AM,0.1,$/kWh
12,NaN,NaN,NaN,NaN,May,0.22,$/kWh,NaN,4AM until 5AM,0.15,$/kWh
13,NaN,NaN,NaN,NaN,June,0.27,$/kWh,NaN,5AM until 6AM,0.2,$/kWh
14,NaN,NaN,NaN,NaN,July,0.24,$/kWh,NaN,6AM until 7AM,0.24,$/kWh
15,NaN,NaN,NaN,NaN,August,0.19,$/kWh,NaN,7AM until 8AM,0.24,$/kWh
16,NaN,NaN,NaN,NaN,September,0.18,$/kWh,NaN,8AM until 9AM,0.26,$/kWh
17,NaN,NaN,NaN,NaN,October,0.15,$/kWh,NaN,9AM until 10AM,0.2,$/kWh


In [14]:
dfc.rename(columns={'Unnamed: 8':'time','Unnamed: 9':'hour_flex', 'Unnamed: 4':'month','Unnamed: 5':'month_flex','Unnamed: 1':'flat_rate'}, inplace=True)
dfc.head()

,Unnamed: 0,flat_rate,Unnamed: 2,Unnamed: 3,month,month_flex,Unnamed: 6,Unnamed: 7,time,hour_flex,Unnamed: 10
8,Flat Rate,0.21,$/kWh,NaN,January,0.2,$/kWh,NaN,12AM until 1AM,0.1,$/kWh
9,NaN,NaN,NaN,NaN,February,0.19,$/kWh,NaN,1AM until 2AM,0.1,$/kWh
10,NaN,NaN,NaN,NaN,March,0.17,$/kWh,NaN,2AM until 3AM,0.1,$/kWh
11,NaN,NaN,NaN,NaN,April,0.18,$/kWh,NaN,3AM until 4AM,0.1,$/kWh
12,NaN,NaN,NaN,NaN,May,0.22,$/kWh,NaN,4AM until 5AM,0.15,$/kWh


In [15]:
fr=dfc.iloc[0,1] # flat rate for common usage
fr

0.21

**Month flex**

In [16]:
mr=dfc[['month','month_flex']].drop(index=range(20,32))
mr['month']=mr['month'].str.slice(0,3)
mr=mr.set_index('month')
mr

,month_flex
month,
Jan,0.2
Feb,0.19
Mar,0.17
Apr,0.18
May,0.22
Jun,0.27
Jul,0.24
Aug,0.19
Sep,0.18


**Hour flex**

In [17]:
hr=dfc[['time','hour_flex']]
hr['time']=hr['time'].str.split(' ').str.slice(0,1).astype(str).str.slice(2,-2)
hr=hr.set_index('time')
hr

,hour_flex
time,
12AM,0.1
1AM,0.1
2AM,0.1
3AM,0.1
4AM,0.15
5AM,0.2
6AM,0.24
7AM,0.24
8AM,0.26


#### Here we have fr,mr,hr in order common, monthly and hourly

In [18]:
rd.head(2)

,watt,month,time,day,weekday,datetime
0,0.384,Mar,3PM,24,Mon,24-Mar-3PM
1,1.201,Aug,5AM,15,NaN,15-Aug-5AM


**Codes to respond to questioned tasks(in the file of tasks)**

In [19]:
print(rd['watt'].mean()) 
# Question 1-B

0.7818779680365298


In [20]:
print(rd[rd['month']=='Feb']['watt'].mean()) 
# Question 2-D

0.8329151785714286


In [21]:
print(rd.groupby(by='weekday').mean().sort_values(by='watt')['watt']) 
# Question 3-A

weekday
Tue    0.727359
Mon    0.731847
Fri    0.760527
Wed    0.777530
Sat    0.801268
Sun    0.844337
Thu    0.867202
Name: watt, dtype: float64


## Making Decision on electricity contract

In [22]:
mr=mr.join(rd.groupby(by='month').sum().sort_values(by='month')[['watt']])
mr['total_flex']=mr['month_flex']*mr['watt']
mr['total_flex'].sum() 
# monthly flex - 1421.82 $    
# Question 5-B 

1421.8209600000025

In [23]:
hr=hr.join(rd.groupby(by='time').sum().sort_values(by='time')[['watt']])
hr['total_flex']=hr['hour_flex']*hr['watt']
hr['total_flex'].sum() 
# hourly total flex - 1369.36 $

1369.3606300000004

In [24]:
rd['watt'].sum()*fr 
# common total flex - 1438.34 $

1438.34271

### According to the results of the final analyst, the "Hourly flex" contract is the best option with desirable and beneficial! Results of the analyst:  
#### 1. Common total payment: 1438.34 
#### 2. Monthly total payment: 1421.83
#### 3. Hourly total payment(best option): 1369.36